# The Battle of Boroughs in Munich

### There are 25 boroughs in Munich. In this notebook i want to compare the housing prices of the boroughs and find out the relationship between the prices and venues. At the end, i try to predict the housing prices of the boroughs in 2021.

### I mainly want to use the data of the venues from foursquare and the data of housing prices which i find in internet.

##### 1. get the venues data from foursquare

import the libraries

In [107]:
import numpy as np
import pandas as pd 
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

get the boroughs in Munich from Wikipedia

In [108]:
res=requests.get('https://de.wikipedia.org/wiki/Stadtbezirke_Münchens')
soup=BeautifulSoup(res.text,'html.parser')
tables=soup.select('table')
df_list=[]
for table in tables:
    df_list.append(pd.concat(pd.read_html(table.prettify())))
df=pd.concat(df_list)
df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Ausländer (%),Dichte (Einw./km²),Einwohner,Fläche (km²),Nr.,Stadt- bezirks- nr.,Stadtbezirk,Stadtbezirksteile (Nr.)
0,261.0,6.708,21.100,315.0,1.0,NaN,Altstadt-Lehel,NaN
1,284.0,11.734,51.644,440.0,2.0,NaN,Ludwigsvorstadt-Isarvorstadt,NaN
2,254.0,11.960,51.402,430.0,3.0,NaN,Maxvorstadt,NaN
3,227.0,15.706,68.527,436.0,4.0,NaN,Schwabing-West,NaN
4,235.0,14.541,61.356,422.0,5.0,NaN,Au-Haidhausen,NaN


In [109]:
df=df.iloc[:25,[6,3]]


In [110]:
df.columns=['Boroughs','Area (km²)']


get the latitudes and longitudes of the boroughs from google map

In [111]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
city ="Maxvorstadt"
country ="Germany"
loc = geolocator.geocode(city+','+ country)
print("latitude is :" ,loc.latitude,"\nlongtitude is:" ,loc.longitude)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


latitude is : 48.1510916 
longtitude is: 11.5624179


In [112]:
from geopy.geocoders import Nominatim
geolocator=Nominatim()
lat=[]
lon=[]
for borough in df['Boroughs']:
    borough=borough
    country="Germany"
    loc=geolocator.geocode(borough+','+country)
    lat.append(loc.latitude)
    lon.append(loc.longitude)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


In [113]:
df['Latitude']=lat
df['Longitude']=lon

In [114]:
df.head()

,Boroughs,Area (km²),Latitude,Longitude
0,Altstadt-Lehel,315.0,48.137828,11.574582
1,Ludwigsvorstadt-Isarvorstadt,440.0,48.130340,11.573366
2,Maxvorstadt,430.0,48.151092,11.562418
3,Schwabing-West,436.0,48.168271,11.569873
4,Au-Haidhausen,422.0,48.128753,11.590536


get venues from foursquare

In [123]:
Client_ID="ZI4AGYQV250W0XUXT0IK015MFBHSRU2WP5DSJHTG1VYCGFRL"
Client_Secret="JBLAAX3KT3N1VWE0I41ZZGW5UXK3R2GKN5KQGSPXIHMK3WIF"
Version=20180605
limit=1000
radius=500
latitude=48.137828
longitude=11.574582

In [124]:
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(Client_ID,Client_Secret,Version,latitude,longitude,radius,limit)

In [185]:
results=requests.get(url).json()

In [188]:
venue=results['response']['groups'][0]['items'][0]['venue']['name']
category=results['response']['groups'][0]['items'][0]['venue']['categories'][0]['name']

In [189]:
category

'Plaza'

In [ ]:
def get_venue_info(lat,lon):
    url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(Client_ID,Client_Secret,Version,latitude,longitude,radius,limit)
    results=requests.get(url).json()
    venue=results['response']['groups'][0]['items'][0]['venue']['name']
    category=results['response']['groups'][0]['items'][0]['venue']['categories'][0]['name']
    
